# Psi4 RT-TDDFT Tutorial

Walk through the socket-based RT-TDDFT coupling using the HCN example.

> **Prerequisites**: install `psi4`, `meep`, and `pymeep`. Ensure the file
> `tests/data/hcn.xyz` is accessible.

## 1. Prepare the Simulation Helper

In [ ]:
import numpy as np
import maxwelllink as mxl
from pathlib import Path

from maxwelllink import sockets as mxs

try:
    import meep as mp
except ImportError as exc:  # pragma: no cover - docs example
    raise RuntimeError("Meep is required. Install via conda-forge.") from exc

XYZ_PATH = Path("tests/data/hcn.xyz").resolve()
if not XYZ_PATH.exists():
    raise FileNotFoundError(f"XYZ file not found: {XYZ_PATH}")


def run_rttddft(until=5):
    host, port = mxs.get_available_host_port()
    hub = mxl.SocketHub(host=host, port=port, timeout=10.0, latency=1e-5)

    molecule = mxl.Molecule(
        hub=hub,
        resolution=10,
        center=mp.Vector3(0, 0, 0),
        size=mp.Vector3(1, 1, 1),
        sigma=0.1,
        dimensions=2,
    )

    sim = mxl.MeepSimulation(
        hub=hub,
        molecules=[molecule],
        cell_size=mp.Vector3(8, 8, 0),
        boundary_layers=[mp.PML(3.0)],
        resolution=10,
        time_units_fs=0.1,
    )

    print(f"SocketHub listening on {host}:{port}")
    driver_cmd = (
        "mxl_driver.py --model rttddft --address {host} --port {port} "
        "--param "molecule_xyz={xyz}, functional=SCF, basis=sto-3g, "
        "delta_kick_au=1e-1, dt_rttddft_au=0.04, electron_propagation=pc, "
        "threshold_pc=1e-6""
    )
    print("Launch the driver in another shell:")
    print(driver_cmd.format(host=host, port=port, xyz=XYZ_PATH))

    sim.run(until=until)

    mu_z = np.array([entry["mu_z_au"] for entry in molecule.additional_data_history])
    time_au = np.array([entry["time_au"] for entry in molecule.additional_data_history])
    return time_au, mu_z


## 2. Run and Compare with the Stored Reference

In [ ]:
time_au, mu_z = run_rttddft()
ref_path = Path("tests/test_psi4_rttddft/test_meep_2d_socket_rttddft_mu_z_au_ref.txt")
reference = np.loadtxt(ref_path)

if not np.allclose(time_au, reference[:, 0], atol=1e-8):
    print("Warning: time axis differs from reference data")

delta = np.max(np.abs(mu_z - reference[:, 1]))
print(f"max deviation = {delta:.3e} a.u.")

import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(time_au, mu_z, label="Simulation")
plt.plot(reference[:, 0], reference[:, 1], label="Reference", linestyle="--")
plt.xlabel("time (a.u.)")
plt.ylabel(r"$\mu_z$ (a.u.)")
plt.legend()
plt.tight_layout()
plt.show()
